In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/NewsTrading/trading_bot

/content/drive/MyDrive/NewsTrading/trading_bot


In [ ]:
!pip install html2text
!pip install datefinder
!pip install -U dask[complete]

  Attempting uninstall: dask
    Found existing installation: dask 2023.8.1
    Uninstalling dask-2023.8.1:
      Successfully uninstalled dask-2023.8.1
  Attempting uninstall: distributed
    Found existing installation: distributed 2023.8.1
    Uninstalling distributed-2023.8.1:
      Successfully uninstalled distributed-2023.8.1


In [ ]:
import dask.dataframe as dd
import dask
import pandas as pd
from dask.distributed import Client
from src.preprocessing.news_parser import filter_body, time, body_formatter

In [ ]:
# client = Client(memory_limit='25GB', processes=False,
#                 n_workers=2, threads_per_worker=1)
# client

In [ ]:
dask.config.set(scheduler="threads")

In [ ]:
input_dir = "data/raw_bzg/"
output_dir = 'data/unraw1_bzg/'

Als erstes müssen wir die HTML-Dokumente zu normalem Text umwandeln, ansonsten sind die Text-Zellen zu groß und führen zu Problemen mit PyArrow/Dask.

In [ ]:
# for year in range(2019, 2020):
#     print(year)
#     df = pd.read_parquet(f"{input_dir}story_df_raw_{year}.parquet")
#     df = dd.from_pandas(df, npartitions=12)
#     df["html_body"] = df["html_body"].apply(body_formatter, meta=pd.Series(dtype="str"))
#     df = df.rename(columns={"html_body":"body"})
#     name_function = lambda x: f"data-{year}-{x}.parquet"
#     df.to_parquet(output_dir, name_function=name_function)

Daten neu partitionieren, sodass alle Partitionen etwa die gleiche Größe haben.

In [ ]:
input_dir = 'data/unraw1_bzg/'
output_dir = 'data/unraw2_bzg/'

In [ ]:
ddf = dd.read_parquet(input_dir+"*.parquet")

In [ ]:
ddf2 = ddf.repartition(npartitions=50)

In [ ]:
name_function = lambda x: f"data-{x}.parquet"
ddf2.to_parquet(output_dir, name_function=name_function)

Ein bisschen die Daten säubern...

In [ ]:
input_dir = 'data/unraw2_bzg/'
output_dir = 'data/unraw3_bzg/'

In [ ]:
ddf = dd.read_parquet(input_dir+"*.parquet")

In [ ]:
# Remove rows for which noo stock ticker is recorded
ddf = ddf[ddf.stocks != '']

In [ ]:
# Convert `channels`  datatype from string to list
ddf["channels"] = ddf["channels"].apply(eval, meta=pd.Series(dtype='object'))

In [ ]:
ddf = ddf.compute()

Untersuche als nächstes die Behauptung, dass PRNewswire und Businesswire den gesamten Markt für Pressemeldungen in den USA kontrollieren. Wenn dem so ist, und sie nicht noch weitere, unwichtige Meldungen veröffentlichen, dann können wir einfach die Newsartikel nach diesen Autoren filtern und uns viel Arbeit ersparen.

In [ ]:
list_of_authors = ddf.author.unique()
list_of_authors

In [ ]:
# value_counts for authors

In [ ]:
# value counts for authors after 2017-05

In [ ]:
# Contains 100k rows
earnings_ddf = ddf[ddf.channels.apply(lambda x: "Earnings" in x)]

In [ ]:
# value counts for authors of earnings reports (contrast to value counts of all news articles)

Deskriptive Statistiken unseres Datensatzes. Haben wir genug Daten, um unser Modell zu trainineren?

In [ ]:
# Around 3k tickers at this moment
russel_tickers = pd.read_pickle("data/tickers.pkl")

In [ ]:
# Filter for stocks in the Russell 3000
# Leaving us with 41k rows
earnings_ddf = earnings_ddf[earnings_ddf.stocks.isin(russel_tickers.categories)]

In [63]:
# Around 140 in 5 years (every 10 days)
dividends = earnings_ddf[earnings_ddf.body.apply(lambda x: "dividend increase " in x)]

Dask Series Structure:
npartitions=1
    string
       ...
Name: author, dtype: string
Dask Name: unique-agg, 4 graph layers